In [8]:
import socket
import time
import math
import sys
import numpy as np
import cv2


In [31]:


# Connect to the server socket
HOST = 'localhost' # Use the local machine as the server
PORT = 65432 # Use the same port number as the server
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.connect((HOST, PORT))
print('Connected to the server.')



Connected to the server.


In [5]:
import sys
import numpy as np
import cv2
import logging
from pygame.math import Vector2
import os
import pygame
from math import sin, radians, degrees, copysign
from pygame.math import Vector2



class Car:
    def __init__(self, x, y, angle=0.0, length=4, max_steering=30, max_acceleration=30.0):
        self.position = Vector2(x, y)
        self.velocity = Vector2(0.0, 0.0)
        self.angle = angle
        self.length = length
        self.max_acceleration = max_acceleration
        self.max_steering = max_steering
        self.max_velocity = 20
        self.brake_deceleration = 10
        self.free_deceleration = 2

        self.acceleration = 0.0
        self.steering = 0.0

    def update(self, dt):
        if self.velocity:
            self.velocity += self.velocity.normalize() * self.acceleration * dt
        else:
            self.velocity += Vector2(1, 1) * self.acceleration * dt


        if self.steering:
            turning_radius = self.length / sin(radians(self.steering))
            angular_velocity = self.velocity.x / turning_radius
        else:
            angular_velocity = 0

        self.position += self.velocity.rotate(-self.angle) * dt
        self.angle += degrees(angular_velocity) * dt


class Game:
    def __init__(self):
        pygame.init()
        pygame.display.set_caption("Car tutorial")
        width = 1280
        height = 720
        self.screen = pygame.display.set_mode((width, height))
        self.clock = pygame.time.Clock()
        self.ticks = 60
        self.exit = False

    def run(self):

        #get current path of notebook
        current_path = os.path.dirname(os.path.abspath("__file__"))
        image_path = os.path.join(current_path, "car.png")
        car_image = pygame.image.load(image_path)
        car = Car(0, 0)
        ppu = 32

        while not self.exit:
            dt = self.clock.get_time() / 100

            # Event queue
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    self.exit = True

            # User input
            pressed = pygame.key.get_pressed()

            if pressed[pygame.K_UP]:
                if car.velocity.x < 0:
                    car.acceleration = car.brake_deceleration
                else:
                    car.acceleration += 1 * dt
            elif pressed[pygame.K_DOWN]:
                if car.velocity.x > 0:
                    car.acceleration = -car.brake_deceleration
                else:
                    car.acceleration -= 1 * dt
            elif pressed[pygame.K_SPACE]:
                if abs(car.velocity.x) > dt * car.brake_deceleration:
                    car.acceleration = -copysign(car.brake_deceleration, car.velocity.x)
                else:
                    car.acceleration = -car.velocity.x / dt
            else:
                if abs(car.velocity.x) > dt * car.free_deceleration:
                    car.acceleration = -copysign(car.free_deceleration, car.velocity.x)
                else:
                    if dt != 0:
                        car.acceleration = -car.velocity.x / dt
            car.acceleration = max(-car.max_acceleration, min(car.acceleration, car.max_acceleration))

            if pressed[pygame.K_RIGHT]:
                car.steering -= 30 * dt
            elif pressed[pygame.K_LEFT]:
                car.steering += 30 * dt
            else:
                car.steering = 0
            car.steering = max(-car.max_steering, min(car.steering, car.max_steering))

            # Logic
            car.update(dt)

            # Drawing
            self.screen.fill((0, 0, 0))
            rotated = pygame.transform.rotate(car_image, car.angle)
            rect = rotated.get_rect()
            self.screen.blit(rotated, car.position * ppu - (rect.width / 2, rect.height / 2))
            pygame.display.flip()

            self.clock.tick(self.ticks)
        pygame.quit()


game = Game()
game.run()

In [39]:
from IPython.display import clear_output
import keyboard


def send_packet(position, theta):
    packet = f'{position[0]},{position[1]},{0},{theta}'.encode('utf-8')
    sock.sendall(packet)


position = np.array([0, 0], dtype=np.float32)
velocity = np.array([0, 0], dtype=np.float32)
theta = 0


while not keyboard.is_pressed('q'):

    if keyboard.is_pressed('w'):
        velocity += np.array([math.cos(theta/180*math.pi), math.sin(theta/180*math.pi)], dtype=np.float32)
    if keyboard.is_pressed('s'):
        velocity -= np.array([math.cos(theta/180*math.pi), math.sin(theta/180*math.pi)], dtype=np.float32)
    if keyboard.is_pressed('a'):
        theta += 1
    if keyboard.is_pressed('d'):
        theta -= 1

    position += velocity * 0.01
    velocity *= 0.9

    clear_output(wait=True)
    print(f'Position: {position}, Velocity: {velocity}, Theta: {theta}')

    #cv2.imshow('image', np.zeros((100, 100, 3)))
    send_packet(position, theta)
    time.sleep(0.01)

cv2.destroyAllWindows() 

Position: [ 1.8579344 20.74566  ], Velocity: [1.0969788e-08 1.2538521e-07], Theta: 265


ConnectionResetError: [WinError 10054] Eine vorhandene Verbindung wurde vom Remotehost geschlossen

In [ ]:


cv2.destroyAllWindows()
# Clean up
packet = b'quit'
sock.sendall(packet)
sock.close()
print('Connection closed.')